In [1]:
import mm_predictor # stat-analysis library
from sklearn import cross_validation, linear_model
import csv
import random
import numpy
import pandas as pd
#from xgboost import XGBClassifier

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# intialize stat & elo dictionaries
mm_predictor.init()

In [3]:
# Load data
season_data = pd.read_csv('./ncaa-data/RegularSeasonDetailedResults.csv')
tourney_data = pd.read_csv('./ncaa-data/NCAATourneyDetailedResults.csv')
tourney_data = tourney_data[tourney_data.Season != 2017]

In [4]:
aggregated_data = pd.concat([season_data, tourney_data])

X,Y = mm_predictor.analyze_teams(aggregated_data)
# Run script

In [35]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import RobustScaler
from sklearn.svm import LinearSVC
from tpot.builtins import StackingEstimator
from xgboost import XGBClassifier

# NOTE: Make sure that the class is labeled 'target' in the data file
# tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
# features = tpot_data.drop('target', axis=1).values
# training_features, testing_features, training_target, testing_target = \
#             train_test_split(features, tpot_data['target'].values, random_state=42)

# Score on the training set was:-0.5443811951009836
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=XGBClassifier(learning_rate=0.01, max_depth=2, min_child_weight=4, n_estimators=100, nthread=1, subsample=0.15000000000000002)),
    RobustScaler(),
    StackingEstimator(estimator=GaussianNB()),
    StackingEstimator(estimator=LinearSVC(C=15.0, dual=True, loss="hinge", penalty="l2", tol=0.0001)),
    StackingEstimator(estimator=GaussianNB()),
    LogisticRegression(C=5.0, dual=False, penalty="l1")
)

exported_pipeline.fit(np.array(X), np.array(Y))


Pipeline(memory=None,
     steps=[('stackingestimator-1', StackingEstimator(estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=2,
       min_child_weight=4, missing=None, n_estimators=100, nthread=1,
       objective='binary:logistic...ty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [36]:
# perform simluation of 2017 NCAA tourney
print("Simulating 2017 NCAA tourney")
submission_data = []
tourney_seeds = pd.read_csv('./ncaa-data/TourneySeeds.csv')
tourney_2017_teams = tourney_seeds[tourney_seeds['Season'] == 2017]['Team']

stat_fields = ['score', 'fga', 'fgp', 'fga3', '3pp', 'ftp', 'or', 'dr',
                   'ast', 'to', 'stl', 'blk', 'pf']
print("Predicting matchups.")
tourney_2017_teams.sort_values()

for team_1 in tourney_2017_teams:
    for team_2 in tourney_2017_teams:
        if team_1 != team_2:
            prediction = mm_predictor.predict_winner(
                team_1, team_2, exported_pipeline, 2017, stat_fields)
            label = '2017_' + str(team_1) + '_' + str(team_2)
            submission_data.append([label, prediction[0][0]])

Simulating 2017 NCAA tourney
Predicting matchups.


In [37]:
teamNames = pd.read_csv('./ncaa-data/Teams.csv')

team_id_map = {}
for index, row in teamNames.iterrows():
    team_id_map[row['TeamID']] = row['TeamName']

teamTeamWinChanceMap = {}
for line in submission_data:
    title = line[0].split('_')
    team1 = team_id_map[int(title[1])]
    team2 = team_id_map[int(title[2])]
    winRate = str(round(line[1]* 100,2))
    if team1 not in teamTeamWinChanceMap:
        teamTeamWinChanceMap[team1] = {}
    if float(winRate) >= 50:
        if team1 in teamTeamWinChanceMap and team2 in teamTeamWinChanceMap[team1] and teamTeamWinChanceMap[team1][team2] > float(winRate):
            continue
        if team2 in teamTeamWinChanceMap and team1 in teamTeamWinChanceMap[team2] and teamTeamWinChanceMap[team2][team1] < float(winRate):
            # update teamTeammap[team2][team1] to remove that key
            teamTeamWinChanceMap[team2].pop(team1, None)
        teamTeamWinChanceMap[team1][team2] = float(winRate)
            
print(teamTeamWinChanceMap)

{'Villanova': {'Duke': 55.51, 'Baylor': 65.71, 'Florida': 64.6, 'Virginia': 67.63, 'South Carolina': 76.75, 'Wisconsin': 65.99, 'Virginia Tech': 69.22, 'Marquette': 75.92, 'Providence': 72.14, 'USC': 72.27, 'UNC Wilmington': 59.84, 'ETSU': 65.15, 'New Mexico St': 66.12, 'Troy': 76.3, "Mt St Mary's": 83.39, 'New Orleans': 82.47, 'Florida St': 63.75, 'West Virginia': 62.21, 'Notre Dame': 62.77, 'Maryland': 69.22, "St Mary's CA": 56.48, 'Northwestern': 72.14, 'Vanderbilt': 72.94, 'VA Commonwealth': 62.49, 'Xavier': 78.12, 'Princeton': 62.21, 'Bucknell': 67.27, 'FL Gulf Coast': 68.93, 'North Dakota': 76.45, 'S Dakota St': 82.66, 'Kansas': 52.78, 'Louisville': 62.35, 'Oregon': 54.0, 'Purdue': 60.96, 'Iowa St': 61.92, 'Creighton': 68.43, 'Michigan': 62.63, 'Miami FL': 69.79, 'Michigan St': 78.24, 'Oklahoma St': 74.18, 'Rhode Island': 61.65, 'Nevada': 58.43, 'Vermont': 56.34, 'Iona': 76.63, 'Jacksonville St': 83.59, 'NC Central': 79.07, 'UC Davis': 80.88, 'North Carolina': 57.6, 'Kentucky': 5

In [38]:
print("Writing %d results." % len(submission_data))
with open('./ncaa-data/submission.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'pred'])
    writer.writerows(submission_data)

Writing 4556 results.


In [39]:
tourney_data2017 = pd.read_csv('./ncaa-data/NCAATourneyDetailedResults.csv')
tourney_data2017 = tourney_data2017[tourney_data2017.Season == 2017]

winners = []

tourney_data_grouped = tourney_data2017[4:].groupby('WTeamID').size().reset_index(name='NumWins')
for index, row in tourney_data_grouped.iterrows():
    teamName = team_id_map[row['WTeamID']]
    wins = row['NumWins']
    if len(winners) == 0:
        winners.append([])
    winners[0].append(teamName)
    if wins > 1:
        if len(winners) == 1:
            winners.append([])
        winners[1].append(teamName)
    if wins > 2:
        if len(winners) == 2:
            winners.append([])
        winners[2].append(teamName)
    if wins > 3:
        if len(winners) == 3:
            winners.append([])
        winners[3].append(teamName)
    if wins > 4:
        if len(winners) == 4:
            winners.append([])
        winners[4].append(teamName)
    if wins > 5:
        if len(winners) == 5:
            winners.append([])
        winners[5].append(teamName)

In [40]:
tourney_data2017

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
914,2017,134,1243,95,1448,88,N,0,31,47,...,27,29,32,7,14,14,11,8,1,25
915,2017,134,1291,67,1309,66,N,0,25,47,...,9,7,12,10,18,12,14,10,2,18
916,2017,135,1413,67,1300,63,N,0,22,48,...,26,10,13,9,21,14,9,10,0,15
917,2017,135,1425,75,1344,71,N,0,24,57,...,20,12,17,7,20,17,9,3,2,21
918,2017,136,1112,100,1315,82,N,0,36,62,...,22,8,16,9,22,18,6,6,2,23
919,2017,136,1139,76,1457,64,N,0,24,49,...,25,5,10,3,24,12,9,6,1,22
920,2017,136,1196,80,1190,65,N,0,26,54,...,24,6,10,13,19,12,17,9,0,19
921,2017,136,1199,86,1195,80,N,0,30,54,...,28,14,20,7,19,18,11,6,1,23
922,2017,136,1211,66,1355,46,N,0,25,63,...,20,5,11,9,29,8,13,3,4,17
923,2017,136,1235,84,1305,73,N,0,30,54,...,26,13,16,10,18,13,7,8,4,14


In [59]:
# First four rows is 'first four' and not in actual bracket

ESPN_MAPPING = {1: 10, 2: 20, 3: 40, 4: 80, 5: 160, 6: 320}

firstFour = tourney_data2017[:4]
mainTourney = tourney_data2017[4:]

tourney = [-1] * max(tourney_seeds['Team'])

# Look at first four teams
for index, row in firstFour.iterrows():
    index1 = int(row['WTeamID'])
    index2 = int(row['LTeamID'])
    team1Name = team_id_map[index1]
    team2Name = team_id_map[index2]
    
    if (team1Name in teamTeamWinChanceMap) and (team2Name in teamTeamWinChanceMap[team1Name]):
        tourney[index2] = index1
        tourney[index1] = tourney[index1] - 1
    else:
        tourney[index1] = index2
        tourney[index2] = tourney[index2] - 1

# MAIN TOURNAMENT 

tourney = [-1] * max(tourney_seeds['Team'])
score = 0
for index, row in mainTourney.iterrows():
    index1 = int(row['WTeamID'])
    index2 = int(row['LTeamID'])
    while tourney[index1] > 0:
        index1 = tourney[index1]
    while tourney[index2] > 0:
        index2 = tourney[index2]
    team1Name = team_id_map[index1]
    team2Name = team_id_map[index2]
    
    if (team1Name in teamTeamWinChanceMap) and (team2Name in teamTeamWinChanceMap[team1Name]): # team1 would win
        tourney[index2] = index1
        tourney[index1] = tourney[index1] - 1
        if team1Name in winners[abs(tourney[index1]) - 2]:
            score += 2**(abs(tourney[index1]) - 2) * 10
            print(team1Name + ' vs ' + team2Name + ', team 1 wins')
            print('Score ' + str(2**(abs(tourney[index1]) - 2) * 10))
        else:
            print('Incorrect: Predicted ' + team1Name + ' vs ' + team2Name + ', team1 wins')
        
    else: #team2 would win
        tourney[index1] = index2
        tourney[index2] = tourney[index2] - 1
        if team2Name in winners[abs(tourney[index2]) - 2]:
            score += 2**(abs(tourney[index1]) - 2) * 10
            print(team1Name + ' vs ' + team2Name + ', team 2 wins')
            print('Score ' + str(2**(abs(tourney[index1]) - 2) * 10))
        else:
            print('Incorrect: Predicted ' + team1Name + ' vs ' + team2Name + ', team2 wins')

            
print(score)

Arizona vs North Dakota, team 1 wins
Score 10
Incorrect: Predicted Butler vs Winthrop, team2 wins
Incorrect: Predicted Florida vs ETSU, team2 wins
Florida St vs FL Gulf Coast, team 1 wins
Score 10
Gonzaga vs S Dakota St, team 1 wins
Score 10
Incorrect: Predicted Iowa St vs Nevada, team2 wins
MTSU vs Minnesota, team 1 wins
Score 10
Incorrect: Predicted Northwestern vs Vanderbilt, team2 wins
Incorrect: Predicted Notre Dame vs Princeton, team2 wins
Incorrect: Predicted Purdue vs Vermont, team2 wins
St Mary's CA vs VA Commonwealth, team 1 wins
Score 10
Villanova vs Mt St Mary's, team 1 wins
Score 10
Incorrect: Predicted Virginia vs UNC Wilmington, team2 wins
West Virginia vs Bucknell, team 1 wins
Score 10
Incorrect: Predicted Wisconsin vs Virginia Tech, team2 wins
Incorrect: Predicted Xavier vs Maryland, team2 wins
Incorrect: Predicted Arkansas vs Seton Hall, team2 wins
Incorrect: Predicted Baylor vs New Mexico St, team2 wins
Cincinnati vs Kansas St, team 1 wins
Score 10
Duke vs Troy, team

In [26]:
teamTeamWinChanceMap

{'Arizona': {'Arkansas': 65.38,
  'Baylor': 66.52,
  'Bucknell': 67.96,
  'Butler': 67.64,
  'Cincinnati': 55.02,
  'Creighton': 69.05,
  'Dayton': 65.05,
  'Duke': 53.95,
  'ETSU': 65.87,
  'FL Gulf Coast': 69.36,
  'Florida': 65.22,
  'Florida St': 64.23,
  'Iona': 76.08,
  'Iowa St': 62.04,
  'Jacksonville St': 82.14,
  'Kansas': 50.34,
  'Kansas St': 78.37,
  'Kent': 73.91,
  'Louisville': 62.55,
  'MTSU': 55.74,
  'Marquette': 75.55,
  'Maryland': 69.66,
  'Miami FL': 70.12,
  'Michigan': 62.89,
  'Michigan St': 77.5,
  'Minnesota': 66.52,
  "Mt St Mary's": 82.04,
  'N Kentucky': 71.61,
  'NC Central': 78.12,
  'Nevada': 57.69,
  'New Mexico St': 67.0,
  'New Orleans': 81.06,
  'North Carolina': 56.63,
  'North Dakota': 75.95,
  'Northwestern': 72.2,
  'Notre Dame': 63.05,
  'Oklahoma St': 74.05,
  'Oregon': 51.96,
  'Princeton': 62.38,
  'Providence': 72.2,
  'Purdue': 60.84,
  'Rhode Island': 61.7,
  'S Dakota St': 81.28,
  'Seton Hall': 68.27,
  'South Carolina': 76.21,
  "St M